In [1]:
import sim
import numpy as np
import math
import time
#PANTALLA
import pygame
import os

pygame.init()
###########
def connect(port):
# Establece la conexión a COPPELIA
# El port debe coincidir con el puerto de conexión en VREP  -- DALE AL PLAY !!!
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

# Conectarse al servidor de COPPELIA
# *** _Hay que ejecutarlo cada vez que se reinicia la simulación ***
clientID = connect(19999)
ret,nema23=sim.simxGetObjectHandle(clientID,'nema23',sim.simx_opmode_blocking)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html
conectado a 19999


In [2]:
MAX_DEG = 60
MIN_DEG = 0
MAX_VEL = 1
STOP = 0

VOL_MIN=50
VOL_MAX=90

RPM_MIN=5
RPM_MAX=30

RADI_MIN=1
RADI_MAX=3

ERROR_MESSAGE= "Valor incorrecte. Siusplau, introdueixi un valor entre"

# Per assigar els parametres del robot, sha utilitzat criteris aproximats, logics i proporcionals als angles maxims i velocitat 
# maxims del robot. PEr fer'ho realista i segur, hauriem de coneixer les caracteristiques mediques dún ventilador manual extern, 
# quines son les necessitats pulmonars del cos humà

def movimentParametritzat(volAire, rpm, radi):
    angle=0 

    maxDeg = volAire-30
 
    vel = ((rpm-RPM_MIN)*(100/(RPM_MAX-RPM_MIN)))/100

    velInsp = vel
    velExp = vel
    
    if radi != RADI_MIN:
        if radi == RADI_MAX:
            velInsp = vel*0.5
        else:
            velExp = vel*0.5  
            
    ## - Funcionalitat - ##
    #for i in range(20):
        while(angle<maxDeg):
            sim.simxSetJointTargetVelocity(clientID, nema23, velInsp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

        while(angle>MIN_DEG):
            sim.simxSetJointTargetVelocity(clientID, nema23, -velExp, sim.simx_opmode_oneshot)
            retCode,radNema = sim.simxGetJointPosition(clientID, nema23, sim.simx_opmode_blocking)
            angle=radNema *180/np.pi

        sim.simxSetJointTargetVelocity(clientID, nema23, STOP, sim.simx_opmode_oneshot)

In [3]:
import random
def getSPO2(aux):
    if aux == 1:#Comportamiento normal
        PSO2 = random.randint(95,99)
    elif aux == 2:
        PSO2 = random.randint(90,99)#Falta de oxigeno
    elif aux == 3:
        PSO2 = random.randint(95,100)#Exceso de oxigeno
    else: 
        print("Valor incorrecto")
        PSO2 = 0
    return PSO2

In [4]:
def setValues():
    vAire = random.randint(50,90)
    RPM = random.randint(15,30)
    radi = random.randint(1,3)
    return vAire,RPM,radi

def funcionOximetre(resNext, perOximetre):
    if (perOximetre < 95):
        if resNext < 40:
            print("Aumentan RPM")
            resNext = resNext + 1
    else:
        if resNext > 10:
            print("Disminuint RPM")
            resNext = resNext - 1
    return resNext

In [5]:
################PANTALLA#############
WIDTH, HEIGHT = 900, 500
WIN = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Respirador Automatic")


BORDERIZQ = pygame.Rect(0, 0, 30, HEIGHT)
BORDERDER = pygame.Rect(WIDTH - 30, 0, 30, HEIGHT)
BORDERALTO = pygame.Rect(0, 0, WIDTH, 30)
BORDERBAJO = pygame.Rect(0, HEIGHT - 30, WIDTH, 30)

GRIS = (128, 128, 128)
WHITE = (255, 255, 255)
PLATEADO = (192, 192, 192)
BLACK = (0, 0, 0)

#FUENTE
myFont = pygame.font.SysFont("Impact", 40)

#Altura, ancho led
LED_WIDTH, LED_HEIGHT = 60, 60
RECUADRO_WIDTH, RECUADRO_HEIGHT = 70, 70
CUADRO_WIDTH, CUADRO_HEIGHT = 600, 70
LOGO_WIDTH, LOGO_HEIGHT = 200, 200

#IMPORT Y TRANSFORM PNG
RED_LED_IMAGE = pygame.image.load(
    os.path.join('images', 'rojo.png'))
RED_LED = pygame.transform.scale(RED_LED_IMAGE, (LED_WIDTH, LED_HEIGHT))
VERDE_LED_IMAGE = pygame.image.load(
    os.path.join('images', 'verde.png'))
VERDE_LED = pygame.transform.scale(VERDE_LED_IMAGE, (LED_WIDTH, LED_HEIGHT))
BLANCO_RECUADRO_IMAGE = pygame.image.load(
    os.path.join('images', 'recuadro-blanco.png'))
BLANCO_RECUADRO = pygame.transform.scale(BLANCO_RECUADRO_IMAGE, (RECUADRO_WIDTH, RECUADRO_HEIGHT))
BLANCO_CUADRO_IMAGE = pygame.image.load(
    os.path.join('images', 'cuadro-blanco.png'))
BLANCO_CUADRO = pygame.transform.scale(BLANCO_CUADRO_IMAGE, (CUADRO_WIDTH, CUADRO_HEIGHT))
LOGO_IMAGE = pygame.image.load(
    os.path.join('images', 'logo.png'))
LOGO = pygame.transform.scale(LOGO_IMAGE, (LOGO_WIDTH, LOGO_HEIGHT))

def draw_window(led, VAire, res, radi, mensaje):
    WIN.fill(PLATEADO)
    pygame.draw.rect(WIN, GRIS, BORDERIZQ)
    pygame.draw.rect(WIN, GRIS, BORDERDER)
    pygame.draw.rect(WIN, GRIS, BORDERALTO)
    pygame.draw.rect(WIN, GRIS, BORDERBAJO)
    if(VAire < 95 or VAire >=100):
        WIN.blit(RED_LED, (led.x, led.y))
        mensaje = "Fallo en volumen de aire"
    else:
        WIN.blit(VERDE_LED, (led.x, led.y))
    VAireDisplay = myFont.render(str(VAire), 1, BLACK)
    resDisplay = myFont.render(str(res), 1, BLACK)
    radiDisplay = myFont.render(str(radi), 1, BLACK)
    MensajeDisplay = myFont.render(str(mensaje), 1, BLACK)
    VolumenDisplay = myFont.render(str("Volumen de Aire"), 1, BLACK)
    ResDisplay = myFont.render(str("RPM"), 1, BLACK)
    RadiDisplay = myFont.render(str("Radi"), 1, BLACK)
    WIN.blit(BLANCO_RECUADRO, (187, 90))
    WIN.blit(BLANCO_RECUADRO, (437, 90))
    WIN.blit(BLANCO_RECUADRO, (637, 90))
    WIN.blit(BLANCO_CUADRO, (87, 250))
    WIN.blit(LOGO, (650, 300))
    WIN.blit(MensajeDisplay, (100, 260))
    WIN.blit(VolumenDisplay, (85, 40))
    WIN.blit(VAireDisplay, (200, 100))
    WIN.blit(ResDisplay, (435, 40))
    WIN.blit(resDisplay, (450, 100))
    WIN.blit(RadiDisplay, (635, 40))
    WIN.blit(radiDisplay, (650, 100))
    pygame.display.update()
    
    led = pygame.Rect(750, 50, LED_WIDTH, LED_HEIGHT)
    mensaje = "Valores normales"
    ############################
    ##########FUNCIO INTRODUIR PARAMETRES (POTENCIOMETRE)################
    vAire, res, radi = setValues()
    #####################################################################
    ##########FUNCIO Get SOP2 (Oximetre)################
    perOximetre = getSPO2(1)# 1- Valors Normals, 2- Falta d'oxigen, 3- Exces d'oxigen
    #####################################################################
    vAireNext = vAire
    resNext = res
    radiNext = radi

    counter = 0
    
    run = True
    surt = True

    while run:
        movimentParametritzat(vAire, res, radi)
        while(vAire == vAireNext and res == resNext and radi == radiNext and surt):
            print(vAire, res, radi)
            print(vAireNext, resNext, radiNext)
            print("Valors normals")
            if(perOximetre < 95 or perOximetre > 99):
                print("Problema d'Oxigen")
                resNext = funcionOximetre(resNext, perOximetre)
                surt = False
                print(resNext)
            movimentParametritzat(vAire, resNext, radi)
            draw_window(led, vAire, res, radi, mensaje)
            perOximetre = getSPO2(2)
        #Si el usuario introdugera nuebos parametros se deverian actualizar en este punto
        vAire = vAireNext
        res = resNext
        radi = radiNext
        surt = True

        if counter == 20:
            run = False
            print(counter)
        else:
            counter += 1
            
    print("Final programa")
        